# DataTuna Final Model Submission
Nora Tombers, Alvaro Prior, Nicolo Prini, Philippe Henderson

Note: This 

In [ ]:
from data_processing import data_processor_full, data_processor_full_jun
from helper_functions import split_function_large, feature_ranker, split_function
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from functools import partial
from scipy.stats.mstats import winsorize


### Supporting Functions


#### Outlier Management:
- Winsorizes columns based on a pre-defined upper and lower outlier threshold (in percentage terms)

In [ ]:
def outlier_management(df, column_list, perc_limit=0.025):
    for column in column_list:
        df[column] = winsorize(df[column], limits=(perc_limit, perc_limit))
    return df


#### addSimpleLags_Diffs
-> This function uses a partial function (_buildLags_Diffs_) to process individual lags. We're essentially bulk applying a single lag (using map) to all columns at once).

- **Inputs**:
    - **df**: DataFrame to modify, 
    - **lag_list**: list of periods of lags or diffs to do (ex. 1 = lag by 1 row, or subtract previous row on current row),
    - **col_list**: list of columns to apply the lagging/diff(ing) to,
    - **change_choice**: either lag, lead, or diff (diff will by default be backwards diff).
- **Output**:
    - Modified DF <- dtypes: int64

In [ ]:
def addSimpleLags_Diffs(df, lag_list, col_list, change_choice='lag'):
    if change_choice == 'lead':
        lag_list = map(lambda x: x * (-1), lag_list)

    arr_lags = list(map(partial(_buildLags_Diffs, df=df,
                                col_list=col_list,
                                change_choice=change_choice),
                        lag_list))

    df = pd.concat([df] + arr_lags, axis=1)

    return df


def _buildLags_Diffs(lag, df, col_list, change_choice):
    if change_choice == 'lag' or change_choice == 'lead':
        return df.groupby('station_code')[col_list].shift(lag).add_suffix(f'_{np.abs(lag)}_{change_choice}')
    elif change_choice == 'diff':
        return df.groupby('station_code')[col_list].diff(lag).add_suffix(f'_{np.abs(lag)}_{change_choice}')

#### data_processor_full - AKA Big Bertha
- So much goes on here. It's late. I'm not sure I have the energy to explain it all....

- TL;DR: Takes in the train + test datasets and calculates all the features, with some help from the supporting functions above. 

In [ ]:
def data_processor_full(train_df, target_df, dep_vars, date_col, date_cols_ToDo,
                        lag_ToDo=[1, 2, 4, 5, 6, 7],
                        diff_ToDo=None,
                        outlier_threshold=0.025, lagChoice='both', cutoff_date = '2021-07-01', verbose=False):

    train_df.sort_values(by=date_col, ascending=True, inplace=True)
    target_df.sort_values(by=date_col, ascending=True, inplace=True)

    LagDiff_vars_ToDo = [x for x in list(train_df.select_dtypes(['int64']).columns) if x not in dep_vars]
    encode_basevars_ToDo = [x for x in list(train_df.select_dtypes(['object']).columns) if
                            x != date_col and x != 'station_code']

    train_df = outlier_management(train_df, LagDiff_vars_ToDo, outlier_threshold)
    target_df = outlier_management(target_df, LagDiff_vars_ToDo, outlier_threshold)

    target_df.drop(columns=target_df.columns[0], axis=1, inplace=True)

    train_target_vars = train_df[['ofd_date', 'station_code'] + dep_vars]
    train_df.drop(columns=dep_vars, inplace=True)

    full_data_to_process = pd.concat([train_df, target_df], ignore_index=True)


    if lagChoice == 'both':
        full_data_to_process = addSimpleLags_Diffs(full_data_to_process, lag_ToDo, LagDiff_vars_ToDo, change_choice='lag')
        full_data_to_process = addSimpleLags_Diffs(full_data_to_process, diff_ToDo, LagDiff_vars_ToDo, change_choice='diff')

    elif lagChoice in ['lag', 'diff', 'lead']:
        full_data_to_process = addSimpleLags_Diffs(full_data_to_process, lag_ToDo, LagDiff_vars_ToDo, change_choice=lagChoice)


    full_data_to_process = full_data_to_process.fillna(0)

    # Encode columns. DC should be excluded from this as we do a special encoding.
    full_data_to_process.drop(columns=encode_basevars_ToDo, inplace=True)


    train_data_processed = full_data_to_process[full_data_to_process.ofd_date < cutoff_date]
    train_data_processed = train_data_processed[train_data_processed.ofd_date > '2021-03-02']
    train_Y = train_target_vars[train_target_vars.ofd_date > '2021-03-02']
    test_data_processed = full_data_to_process[full_data_to_process.ofd_date >= cutoff_date]


    train_Y['ofd_date'] = train_Y.ofd_date.astype('datetime64[ns]')
    train_data_processed['ofd_date'] = train_data_processed.ofd_date.astype('datetime64[ns]')
    train_Y['station_code'] = train_Y.station_code.astype(str)
    train_data_processed['station_code'] = train_data_processed.station_code.astype(str)

    final_train_all = pd.merge(train_data_processed, train_Y, on=["ofd_date", 'station_code'], how="left")

    final_train_all['station_code'] = final_train_all['station_code'].apply(lambda x: int(x[1:]))

    test_data_processed['station_code'] = test_data_processed['station_code'].apply(lambda x: int(x[1:]))


    final_train_all.reset_index(drop=True, inplace=True)
    train_data_processed.reset_index(drop=True, inplace=True)
    train_Y.reset_index(drop=True, inplace=True)
    test_data_processed.reset_index(drop=True, inplace=True)
    
    if verbose:
        for column in list(full_data_to_process.columns):
            print(column)
    else:
        print("Data processing is done.")
        print(f"We now have {len(list(full_data_to_process.columns))} features. Woop woop.")


    return final_train_all, test_data_processed

#### Training Data Splitting functions
- **split_function:** will just split train data into two splits: train + test
- **split_function_large:** will split train data into two training datasets + one test. This is used for the feature selection function.

In [ ]:
def split_function(df, target_var, train_perc=.8):
    all_dates_unique = df['ofd_date'].unique()

    cutoff_date = all_dates_unique[int(len(all_dates_unique) * train_perc)]

    train_data = df[df.ofd_date < cutoff_date]
    test_data = df[df.ofd_date >= cutoff_date]

    train_data.set_index(['ofd_date'], inplace=True)
    test_data.set_index(['ofd_date'], inplace=True)

    train_X, train_Y = train_data.drop(target_var, axis=1), train_data[target_var]
    test_X, test_Y = test_data.drop(target_var, axis=1), test_data[target_var]

    return train_X, train_Y, test_X, test_Y

In [ ]:
def split_function_large(df, target_var, train_perc1=.7, train_perc2=.2):
    all_dates_unique = df['ofd_date'].unique()

    cutoff_date_one = all_dates_unique[int(len(all_dates_unique) * train_perc1)]
    cutoff_date_two = all_dates_unique[int(len(all_dates_unique) * (train_perc1 + train_perc2))]

    train_data_one = df[df.ofd_date < cutoff_date_one]
    mask = ((df['ofd_date'] >= cutoff_date_one) & (df['ofd_date'] < cutoff_date_two))
    train_data_two = df.loc[mask]
    test_data = df[df.ofd_date >= cutoff_date_two]

    train_data_one.set_index(['ofd_date'], inplace=True)
    train_data_two.set_index(['ofd_date'], inplace=True)
    test_data.set_index(['ofd_date'], inplace=True)

    train_X_one, train_Y_one = train_data_one.drop(target_var, axis=1), train_data_one[target_var]
    train_X_two, train_Y_two = train_data_two.drop(target_var, axis=1), train_data_two[target_var]
    test_X, test_Y = test_data.drop(target_var, axis=1), test_data[target_var]

    return train_X_one, train_Y_one, train_X_two, train_Y_two, test_X, test_Y

## Feature Selection Functions

return_columns: Just searches for the features with a given importance weight.

In [ ]:
def return_columns(dict, threshold):
    column_list = []
    for feature in dict.keys():
        if dict[feature] >= threshold:
            column_list.append(feature)
        else:
            continue
    return column_list

#### cross_val_test
- Performs the RMSE estimation (averaged out over nr_cross rounds) for feature selection.
- Takes two training sets (second one is used as the eval_set), and a test_set to predict RMSE of the selected features

In [ ]:
def cross_val_test(nr_cross, select_X_train_one, y_train_one, select_X_train_two, y_train_two, select_X_test, y_test):
    all_RMSE = []
    for _ in range(nr_cross):
        selection_model = xgb.XGBRegressor(n_estimators=250)
        selection_model.fit(select_X_train_one, y_train_one,
                            eval_set=[(select_X_train_one, y_train_one), (select_X_train_two, y_train_two)],
                            early_stopping_rounds=20,
                            verbose=False)

        preds2 = pd.DataFrame(selection_model.predict(select_X_test))
        RMSE = mean_squared_error(y_test, preds2, squared=False)
        all_RMSE.append(RMSE)

    average_RMSE = sum(all_RMSE) / len(all_RMSE)
    return average_RMSE

#### feature_ranker
- Will run a first XGBoost round to get all feature weights, then creates a dict of the features & their scores. 
- Then we loop through each distinct feature score (in ascending order) and progressively filter out features. 
- RMSE cross validation is done with the **cross_val_test** function, and the features yielding the best average RMSE are returned to the main model. 

In [ ]:
def feature_ranker(x_train_one, y_train_one, x_train_two, y_train_two, x_test, y_test, target_var):
    xgb_reg = xgb.XGBRegressor(n_estimators=1000)
    xgb_reg.fit(x_train_one, y_train_one, eval_set=[(x_train_one, y_train_one), (x_train_two, y_train_two)],
                early_stopping_rounds=50,
                verbose=False)

    importances = xgb_reg.get_booster().get_score(importance_type='weight')

    importances_scores = [x for x in importances.values()]

    new_scores = []
    for score in importances_scores:
        if score not in new_scores:
            new_scores.append(score)

    new_scores.sort()

    preds = pd.DataFrame(xgb_reg.predict(x_test))
    RMSE_baseline = mean_squared_error(y_test, preds, squared=False)
    print(f"Base RMSE for {target_var}, All features: {RMSE_baseline}")

    results = {}
    for score in new_scores:

        features_to_use = return_columns(importances, score)
        if 'station_code' not in features_to_use:
            features_to_use = features_to_use + ['station_code']
        select_X_train_one = x_train_one[features_to_use]
        select_X_train_two = x_train_two[features_to_use]
        select_X_test = x_test[features_to_use]

        avg_RMSE = cross_val_test(5, select_X_train_one=select_X_train_one, y_train_one=y_train_one,
                                  select_X_train_two=select_X_train_two, y_train_two=y_train_two,
                                  select_X_test=select_X_test, y_test=y_test)

        results[avg_RMSE] = features_to_use
        print(
            f"Feature Sel. RMSE for {target_var}, Score Threshold: {score}, Total features: {len(features_to_use)}, Average RMSE: {avg_RMSE}")

    lowest_RMSE = min([x for x in results.keys()])
    optimal_params = results[lowest_RMSE]
    print(f"Lowest RMSE: {lowest_RMSE} vs. Baseline: {RMSE_baseline}, with {len(optimal_params)} features: {optimal_params} ")
    return optimal_params

## Model Tuning + Prediction

#### The Manager
- Manages tasks detailed above: Splitting data using split_function_large, then finding best parameters, then transforming the dataset.
- Once that is done, it will then run xGBGrid_FIND

In [ ]:
def model_manager_features(full_df_train, target_var, test_data, non_target_var, params):
    print(f"Working on: {target_var}")
    full_df_train.drop(columns=[non_target_var], axis=1, inplace=True)
    train_X_one, train_Y_one, train_X_two, train_Y_two, test_X, test_Y = split_function_large(df=full_df_train,
                                                                                              target_var=target_var,
                                                                                              train_perc1=0.7,
                                                                                              train_perc2=0.2)

    optimal_features = feature_ranker(x_train_one=train_X_one, y_train_one=train_Y_one, x_train_two=train_X_two,
                                      y_train_two=train_Y_two, x_test=test_X, y_test=test_Y, target_var=target_var)


    full_train_df_selected = full_df_train[['ofd_date'] + optimal_features + [target_var]]
    print(full_train_df_selected)
    full_test_selected = test_data[['ofd_date'] + optimal_features]
    print(full_test_selected)

    train_X, train_Y, validation_X, validation_Y = split_function(df=full_train_df_selected,
                                                                  target_var=target_var,
                                                                  train_perc=0.8)

    optimal_params, final_prediction = xGBGrid_FIND(x_train=train_X, y_train=train_Y, x_validate=validation_X,
                                                    y_validate=validation_Y, x_test=full_test_selected,
                                                    params=params, target_var=target_var)

    return optimal_features, optimal_params, final_prediction

#### xGBGrid_FIND
- Runs HalvingGridSearchCV (instead of GridSearchCV) based on params defined below. 
- Once optimal model params are found, it will fit to the test data. 
- Finally, it cleans the predictions and returns it in a nice format for the final prediction data.

**Note:** A sharp eye will notice that n_jobs is set pretty high. Don't set it higher, and in fact, set it lower. At 400 I crashed a M1 Max Mac. 

In [ ]:
def xGBGrid_FIND(x_train, y_train, x_validate, y_validate, x_test, params, target_var):
    xgb_reg = xgb.XGBRegressor()
    grid = HalvingGridSearchCV(xgb_reg, param_grid=params, factor=3, scoring='neg_mean_squared_error', n_jobs=300,
                               verbose=3)
    # grid = GridSearchCV(xgb_reg, params, scoring=scoring_func, n_jobs=-1, cv=5, verbose=3)
    grid.fit(x_train, y_train, eval_set=[(x_validate, y_validate)], early_stopping_rounds=20, verbose=False)
    gridcv_xgb = grid.best_estimator_
    print(f"Best Params: {gridcv_xgb}")


    x_base = x_test[['ofd_date', 'station_code']]

    x_test = x_test[list(x_train.columns)]
    preds = gridcv_xgb.predict(x_test)
    preds_df = pd.DataFrame(preds)

    final_pred = convert_to_final(x_base, preds_df, target_var)
    return gridcv_xgb, final_pred

## Supporting functions to prediction -> Just to clean predictions into right format

In [ ]:
def convert_to_final(test_X, preds, var):
    colname = 'yhat_' + str(var)
    preds[colname] = preds[0]
    test_X = test_X.reset_index()

    full_prediction = test_X.join(preds)
    print(full_prediction)
    full_prediction = full_prediction[['ofd_date', 'station_code', colname]]
    return full_prediction

In [ ]:
def final_merger(mnr_df, earlies_df):
    mnr_df.reset_index(drop=True, inplace=True)
    earlies_df.reset_index(drop=True, inplace=True)

    forecast_full = pd.merge(mnr_df, earlies_df, how="left", on=["ofd_date", "station_code"])
    forecast_full['Expected'] = forecast_full['yhat_Earlies_Exp'] - forecast_full['yhat_MNR_SNR_Exp']

    forecast_full['Id'] = forecast_full['ofd_date'].astype(str) + "_D" + forecast_full['station_code'].astype(str)

    forecast_final_result = forecast_full[['Id', 'Expected']]
    return forecast_final_result

## Main Processing Function

### Input Parameters

We create a lot of lags and diff (day0 - day(-1) ) columns to provide the model with information as to past events.

In [2]:
target_vars = [['MNR_SNR_Exp', 'Earlies_Exp'], ['Earlies_Exp', 'MNR_SNR_Exp']]
base_date_col = 'ofd_date'


lag_to_do = []
for x in range(1, 31):
    lag_to_do.append(x)
    
diff_to_do = []
for x in range(1, 22):
    diff_to_do.append(x)

#### Parameters for HalvingGridSearchCV

Many parameters. Hence using HalvingGridSearchCV

In [ ]:
xGparams_high = {
        'learning_rate': [0.05, 0.1, 0.2, 0.3],
        'n_estimators': [500, 700],
        'min_child_weight': [4, 5],
        'gamma': [i / 10.0 for i in range(4, 6)],
        'subsample': [i / 10.0 for i in range(8, 11)],
        'colsample_bytree': [i / 10.0 for i in range(8, 11)],
        'max_depth': [3, 4, 5]
    }

#### Big Momma Part II
- Loops through the two base target variables (MNR_SNR_Exp and Earlies_Exp).
- Preps the data, then runs the model_manager to select best features, find best parameters, and make the prediction

In [ ]:
# Run data processor
results_dict = {}

result_dfs = []

synth_tests = []

early_testing = False

for i in range(len(target_vars)):
    train_data = pd.read_csv('./train_data.csv', sep=',')
    test_data = pd.read_csv('./test.csv', sep=',')


    final_all_train, test_data_processed = data_processor_full(
        train_df=train_data,
        target_df=test_data,
        dep_vars=['Earlies_Exp', 'MNR_SNR_Exp'],
        lag_ToDo=lag_to_do,
        diff_ToDo=diff_to_do,
        outlier_threshold=0.025,
        lagChoice='both',
        cutoff_date='2021-07-01',
        verbose=False)

    optimal_features, optimal_params, result_prediction = model_manager_features(full_df_train=final_all_train,
                                                                                 target_var=target_vars[i][0],
                                                                                 test_data=test_data_processed,
                                                                                 non_target_var=target_vars[i][1], 
                                                                                 params=xGparams_low)



    results_dict[target_vars[i][0]] = {}
    results_dict[target_vars[i][0]]['best_features'] = optimal_features
    results_dict[target_vars[i][0]]['best_param'] = optimal_params

    result_dfs.append(result_prediction)

final_output = final_merger(result_dfs[0], result_dfs[1])
print(f"Final best params for prediction: {results_dict}")

print(final_output)
final_output.to_csv('phil_submission_final_squared.csv', index=False)